# Install prerequisites

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
csv_path = '/IK_rr_DataFrame.csv'
df = pd.read_csv(csv_path)
# A DataFrame containing the parsed text for all research articles

In [ ]:
df.head()

,FileName,Author,Text
0,Controlled synthesis.pdf,Amit Saxena,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,High Oxygen Nanocomposite.pdf,Amit Saxena,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,Cutting Edge.pdf,Amit Saxena,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,Maternal and Fetal Factors.pdf,Amit Saxena,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,Opposing Actions of Fibroblast.pdf,Amit Saxena,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


# Input the TEST PDF

In [ ]:
import PyPDF2
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import unicodedata
import re

#Test pdf for evaluation
pdf_path = "/Dataset-IK/Amit Saxena/A Review of Clustering Techniques.pdf"
text_column = "PDF"

k = 5 #Retrieve top k authors

stop_words = stopwords.words('english')
stop_words.extend(['@',"'",'.','"','/','!',',',"'ve","...","n't",'$',"'s",'©','''"''',"''",'..','&','*',';','”','``',':','#','!','-','–','?','%',"'d","'m",'+','++','(',')','()'])
stop_words = set(stop_words)

In [ ]:
df.shape

(16, 3)

# Defining functions for subtasks

In [ ]:
def preprocess_test_document(pdf_path):
    test_document_text = extract_text_from_pdf(pdf_path)
    preprocessed_text = preprocess_text(test_document_text)
    return preprocessed_text

def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub('<[^>]*>', '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = [word for word in text if len(word) > 2]
    text = ' '.join(text)
    return text

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            pdf_text = ''
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                pdf_text += page.extract_text()
    except KeyError as e:
        print(f"Error processing file '{pdf_path}': {e}")
        return ''
    return pdf_text

def preprocess_csv_file(csv_path, text_column):
    data = pd.read_csv(csv_path)
    csv_text_data = data[text_column].tolist()
    preprocessed_texts = [preprocess_text(text) for text in csv_text_data]
    return preprocessed_texts

In [ ]:
pdf_path #Path to a sample PDF file for matching

'/Dataset-IK/Amit Saxena/A Review of Clustering Techniques.pdf'

In [ ]:
test_text_sw_removed = preprocess_test_document(pdf_path)
test_text_sw_removed

'review clustering techniques developments amit saxena mukesh prasad akshansh gupta neha bharill prakash patel aruna tiwari meng joo weiping ding chin teng lin department computer science guru ghasidas vishwavidyal aya bilaspur india centre artificial intelligence university technology sydney sydney australia school computational integrative sciences jawaharlal nehru university new delhi india department computer science engineering ndian institute technology indore india school electrical electronic engineering nanyang technological university singapore school computer technology nantong university nantong china abstract paper presents comprehensive study clustering exiting methods developments made various times clustering defined unsupervised learning objects grouped basis similarity inherent among different methods clustering objects hierarchical partitional grid density based model based approaches used methods discussed respective state art applicability measures similarity well 

In [ ]:
csv_path

'/IK_rr_DataFrame.csv'

In [ ]:
csv_txts_sw_removed = preprocess_csv_file(csv_path, 'Text')

In [ ]:
len(csv_txts_sw_removed) #Stopwords have been removed from the text of each file

16

# Finding similarity using plaintext

In [ ]:
#Let's find the most similar document based on jaccard simialrity (plain text)

def Jaccard_Similarity(doc1, doc2):

    # List the unique words in a document
    words_doc1 = set(doc1.lower().split())
    words_doc2 = set(doc2.lower().split())

    ## doc1 = This is a cat
    ## doc2 = This is a dog

    ## words_doc1: {This, is, a, cat}
    ## words_doc2: {This, is, a, dog}
    # Find the intersection of words list of doc1 & doc2
    intersection = words_doc1.intersection(words_doc2)
    # Intersection: {This, is, a}
    # Union: {This, is, a, cat, dog}

    # Find the union of words list of doc1 & doc2
    union = words_doc1.union(words_doc2)

    # Calculate Jaccard similarity score
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)



jac_sim = np.zeros(df.shape[0]) #Initializing jac_sim of every document with 0
for i in range(len(csv_txts_sw_removed)):
    jac_sim[i] = Jaccard_Similarity(test_text_sw_removed, csv_txts_sw_removed[i])

In [ ]:
#Finding index of maximum jac_sim
np.argmax(jac_sim)

10

In [ ]:
#Details of the most similar document
df.loc[np.argmax(jac_sim)]

FileName                A Review of Clustering Techniques.pdf
Author                                            Amit Saxena
Text        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
Name: 10, dtype: object

# Finding simialrity using BoW

In [ ]:
# We will do stemming to reduce the vocabulary.
# Stemming is preferred over lemmatization for simialrity based tasks.
from nltk.stem import SnowballStemmer
sb_stemmer = SnowballStemmer('english')

stemmed_csv_data = []
for each_text in csv_txts_sw_removed:
    stemmed_txt = []
    for each_word in each_text.split(' '):
        stemmed_txt.append(sb_stemmer.stem(each_word))
    stemmed_csv_data.append(stemmed_txt)

In [ ]:
len(stemmed_csv_data) #Stemmed text of the PDFs

16

In [ ]:
#Stemmed test PDF
stemmed_test_txt = []
for each_word in test_text_sw_removed.split():
    stemmed_test_txt.append(sb_stemmer.stem(each_word))

### Generating BoW representation (Presence/Absence - 1/0)

In [ ]:
# Combining test PDF text with csv text (to get same representation)
stemmed_csv_data.append(stemmed_test_txt)
# You can also use the encoder fit on the csv_text on the text PDF text.

In [ ]:
len(stemmed_csv_data)

17

In [ ]:
#This expects the input as an array of lists or as a list of lists
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
csv_data = pd.DataFrame(te.fit(stemmed_csv_data).transform(stemmed_csv_data).astype('int'), columns= te.columns_)

In [ ]:
csv_data.shape #Enormous vocabulary of 8580 words

(17, 8580)

In [ ]:
#Using frequent features only to process the similarities quickly
col_lst_less_than_10 = []
for col in csv_data.columns:
    if csv_data[col].sum() < 10:
        col_lst_less_than_10.append(col)

In [ ]:
len(col_lst_less_than_10)

8297

In [ ]:
csv_data_rm_10 = csv_data.drop(col_lst_less_than_10, axis = 1)
csv_data_rm_10.shape

(17, 283)

In [ ]:
# Let's compute cosine similarity. You can also try jaccard similarity here.
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
def calculate_similarity(test_rep, csv_rep):
    similarity_scores = []
    indices = []
    for i, csv_reps in enumerate(csv_rep):
        similarity =  cosine_similarity(np.array(test_rep).reshape(1,-1),np.array(csv_reps).reshape(1,-1))[0][0]
        similarity_scores.append(similarity)
        indices.append(i)
    return similarity_scores, indices

In [ ]:
sim_scores, ind = calculate_similarity(csv_data.iloc[csv_data.shape[0] - 1].values, csv_data.loc[0:csv_data.shape[0]-2].values)

In [ ]:
np.argmax(sim_scores)

10

In [ ]:
#Details of the most similar document
df.loc[np.argmax(jac_sim)]

FileName                A Review of Clustering Techniques.pdf
Author                                            Amit Saxena
Text        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
Name: 10, dtype: object

#### Retrieve top-k unique authors

In [ ]:
def retrieve_top_k_unique_authors(df, similarity_scores, indices, k):
    # Sort scores and corresponding indices
    sorted_scores_indices = sorted(zip(similarity_scores, indices), key=lambda x: x[0], reverse=True)
    unique_authors = []

    # Loop over sorted scores and indices
    for _, idx in sorted_scores_indices:
        # Get author corresponding to index
        author = df.iloc[idx]['Author']
        # Append author to unique_authors list if it's not there yet
        if author not in unique_authors:
            unique_authors.append(author)
            # Break the loop if we have found k unique authors
            if len(unique_authors) == k:
                break

    return unique_authors

In [ ]:
k=3
top_k_authors = retrieve_top_k_unique_authors(df, sim_scores, ind, k)
top_k_authors

['Amit Saxena']

# Using Term Frequency Representation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# The expected input is list of strings or an array of strings
str_text = []
for each_text in stemmed_csv_data:
    mystr = ''
    for each_word in each_text:
        mystr = mystr + ' ' + each_word #You can achieve the same using join
    str_text.append(mystr.lstrip(' '))

In [ ]:
cv = CountVectorizer()
cv.fit(str_text)
tf_data = pd.DataFrame(cv.transform(str_text).todense(), columns = sorted(cv.vocabulary_))
tf_data.head()

,aaai,aalexa,aar,abaci,abbrevi,abcam,abcellera,abdalla,abdallah,abdomin,...,zou,zro,zucker,zugaib,zurakowski,zutphen,zyme,zymek,zymographi,zymosan
0,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
4,0,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
#Using frequent features only to process the similarities quickly
col_lst_less_than_10 = []
for col in tf_data.columns:
    if tf_data[col].sum() < 10:
        col_lst_less_than_10.append(col)

In [ ]:
len(col_lst_less_than_10)

7176

In [ ]:
tf_data.drop(col_lst_less_than_10, axis = 1, inplace = True)
tf_data.shape

(17, 1403)

In [ ]:
# Calculate similarity between the test document and CSV documents
similarity_scores, indices = calculate_similarity(tf_data.iloc[tf_data.shape[0] - 1].values, tf_data.loc[0:tf_data.shape[0]-2].values)

# Load the CSV data
df = pd.read_csv(csv_path)

# Retrieve top-k unique authors
k=5
top_k_authors = retrieve_top_k_unique_authors(df, similarity_scores, indices, k)

# Print the top-k authors
print(top_k_authors)

['Amit Saxena']


# Using TF-IDF Representation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
fidf = TfidfVectorizer()
fidf.fit(str_text)
tf_idf_data = pd.DataFrame(fidf.transform(str_text).todense(), columns = sorted(fidf.vocabulary_))
tf_idf_data.shape

(17, 8579)

In [ ]:
# Calculate similarity between the test document and CSV documents
similarity_scores, indices = calculate_similarity(tf_idf_data.iloc[tf_idf_data.shape[0] - 1].values, tf_idf_data.loc[0:tf_idf_data.shape[0]-2].values)

# Load the CSV data
df = pd.read_csv(csv_path)

# Retrieve top-k unique authors
k=5
top_k_authors = retrieve_top_k_unique_authors(df, similarity_scores, indices, k)

# Print the top-k authors
print(top_k_authors)

['Amit Saxena']


# Validating Performance

In [ ]:
def top_k_accuracy(y_true, y_pred, k=1):
    """
    Compute the top-k accuracy based on word presence in predictions.

    Parameters:
    y_true (str): True label (actual author).
    y_pred (list): List of predicted authors for the test PDFs.
                   The shape of y_pred should be (num_PDFs, k).
    k (int): The value of k for top-k accuracy. Default is 1 (top-1 accuracy).

    Returns:
    top_k_acc (float): The top-k accuracy value.
    """
    if k <= 0 or not isinstance(k, int):
        raise ValueError("k must be a positive integer.")

    num_samples = len(y_pred)
    num_correct = 0

    for i in range(num_samples):
        if y_true in y_pred[i][:k]:
            num_correct += 1

    top_k_acc = num_correct / num_samples
    return top_k_acc

In [ ]:
# Top-3 accuracy with an example
top_k_accuracy('Sanatan',[['Sanatan','Arash','Abhinav'],['Arash','Abhinav','Vartika'],['Arash','Sanatan','Abhinav']],3)

0.6666666666666666

In [ ]:
# Top-1 accuracy (Matching with the first prediction only)
top_k_accuracy('Sanatan',[['Sanatan','Arash','Abhinav'],['Arash','Abhinav','Vartika'],['Arash','Sanatan','Abhinav']],1)

0.3333333333333333

## Some pointers for unbiased evaluation
> You can split the given dataset into train and test, and evaluate the performance of your models with Top-k accuracy **for different values of k**.


* When dividing the data, make sure that at least one PDF file from each author appears in both the training and testing sets. Exclude authors for whom there is only one PDF available in the dataset.

* You can average the performance over multiple runs.

* You can think of aggregating and comparing documents as a whole (per author).

